In [27]:
#!pip install --quiet glvis
import numpy as np
import matplotlib.pyplot as plt
from  bfp import *
import mfem.ser as mfem
#from mfem.common.arg_parser import ArgParser
from glvis import glvis, GlvisData
import seaborn as sns
import pandas as pd
import os

In [28]:
class EDependentCoefficient(mfem.PyCoefficient):
    def __init__(self, xs_t_data, E_start, E_end):
        super(EDependentCoefficient, self).__init__()
        self.E_start = E_start
        self.E_end = E_end
        if isinstance(xs_t_data, (int, float)):
            self.constant = True
            self.constant_value = float(xs_t_data)
        else:
            self.constant = False
            self.xs_t_data = xs_t_data
            self.n_groups = len(xs_t_data)
            self.E_bins = np.linspace(E_start, E_end, self.n_groups + 1)

    def EvalValue(self, x):
        if self.constant:
            return self.constant_value
        E = x[1]
        for i in range(self.n_groups - 1):
            if np.isclose(E, self.E_bins[i+1]):
                return float(self.xs_t_data[i+1])
            if E <= self.E_bins[i] and E > self.E_bins[i+1]:
                return float(self.xs_t_data[i])
        return float(self.xs_t_data[-1])

class XDependentCoefficient(mfem.PyCoefficient):
    def __init__(self, xs_t_data, x_start, x_end):
        super(XDependentCoefficient, self).__init__()
        self.x_start = x_start
        self.x_end = x_end
        if isinstance(xs_t_data, (int, float)):
            self.constant = True
            self.constant_value = float(xs_t_data)
        else:
            self.constant = False
            self.xs_t_data = xs_t_data
            self.n_groups = len(xs_t_data)
            self.x_bins = np.linspace(x_start, x_end, self.n_groups + 1)

    def EvalValue(self, x):
        if self.constant:
            return self.constant_value

        x_val = x[0]
        for i in range(self.n_groups - 1):
            if np.isclose(x_val, self.x_bins[i+1]):
                return float(self.xs_t_data[i+1])
            if self.x_bins[i] <= x_val < self.x_bins[i+1]:
                return float(self.xs_t_data[i])
        return float(self.xs_t_data[-1])

class MyVectorCoefficient(mfem.VectorConstantCoefficient):
    def __init__(self, mu_coeff, S_coeff):
        super(MyVectorCoefficient, self).__init__(2)
        self.mu_coeff = mu_coeff 
        self.S_coeff = S_coeff

    def EvalValue(self, x):
        mu_val = self.mu_coeff.EvalValue(x) 
        S_val = self.S_coeff.EvalValue(x) 
        return [mu_val, S_val]
    
class InflowCoefficientSN(mfem.PyCoefficient):
    def __init__(self, inflow, mu):
        super(InflowCoefficientSN, self).__init__()
        self.inflow = inflow
        self.mu = mu
    def EvalValue(self, x):
        if self.mu > 0:
            return self.inflow
        else:
            return 0
        
class ConstantVector(mfem.VectorPyCoefficient):
    def __init__(self, constant):
        super(ConstantVector, self).__init__(1)
        self.constant = constant 
    def EvalValue(self, x):
        return self.constant

In [29]:
def test_EDependentCoefficient():
    xs_t_data = [0,1,2,3,4]
    coeff = EDependentCoefficient(xs_t_data, 1, 0.01)
    print(coeff.EvalValue([0.0, 1]))
    print(coeff.EvalValue([0.0, 0.802]))
    print(coeff.EvalValue([0.0, 0.604]))
    print(coeff.EvalValue([0.0, 0.406]))
    print(coeff.EvalValue([0.0, 0.208]))
    print(coeff.EvalValue([0.0, 0.01]))

def test_EDependentCoefficient_Constant():
    coeff = EDependentCoefficient(40, 0, 1)
    print(coeff.EvalValue([0.0, 1]))
    print(coeff.EvalValue([0.0, 0.802]))
    print(coeff.EvalValue([1.0, 0.604]))
    print(coeff.EvalValue([0.3, 0.406]))
    print(coeff.EvalValue([0.6, 0.208]))
    print(coeff.EvalValue([0.8, 0.01]))

#print("Test for E dependent Coefficient")
#test_EDependentCoefficient()
#print("Test for E dependent Constant Coefficient")
#test_EDependentCoefficient_Constant()

def test_XDependentCoefficientt():
    xs_t_data = [0,1,2,3,4]
    coeff = XDependentCoefficient(xs_t_data, 0, 1)
    print(coeff.EvalValue([0.0, 0.00]))
    print(coeff.EvalValue([0.2, 0.00]))
    print(coeff.EvalValue([0.4, 0.00]))
    print(coeff.EvalValue([0.6, 0.00]))
    print(coeff.EvalValue([0.8, 0.00]))
    print(coeff.EvalValue([1.0, 0.00]))

def test_XDependentCoefficient_Constant():
    coeff = XDependentCoefficient(80, 0.0, 1.0)
    print(coeff.EvalValue([0.0, 0.00]))
    print(coeff.EvalValue([0.2, 0.00]))
    print(coeff.EvalValue([0.4, 0.00]))
    print(coeff.EvalValue([0.6, 0.00]))
    print(coeff.EvalValue([0.8, 0.00]))
    print(coeff.EvalValue([1.0, 0.00]))

#print("Test for X dependent Coefficient")
#test_XDependentCoefficientt()
#print("Test for X dependent Constant Coefficient")
#test_XDependentCoefficient_Constant()

def test_MyVectorCoefficient():
    mu_data = [5]
    mu = XDependentCoefficient(mu_data, 0.0, 1.0)
    S_data = [0,1,2,3,4]
    S = EDependentCoefficient(S_data, 1.0, 0.01)
    coeff = MyVectorCoefficient(mu, S)
    print(coeff.EvalValue([0.0, 0.802]))
    print(coeff.EvalValue([0.2, 0.604]))
    print(coeff.EvalValue([0.4, 0.406]))
    print(coeff.EvalValue([0.6, 0.208]))
    print(coeff.EvalValue([0.8, 0.01]))

#print("Test for Vector Coefficient")
#test_MyVectorCoefficient()


In [30]:
nx = 10
nE = 10
x_start = 0.0
x_end = 1.0
E_start = 0.0
E_end = 1.0
N_ang = 4
order = 1
inflow = 20.0

#mesh = create_2D_mesh(nx, nE, x_start, x_end, E_start, E_end)
mesh = mfem.Mesh(nx, nE, "QUADRILATERAL", True, x_end, E_end)
#for x in range(ref_levels):
mesh.UniformRefinement()
#mesh.UniformRefinement()
#mesh.UniformRefinement()
#mesh.UniformRefinement()
dim = mesh.Dimension()

# btype = 0-GaussLegendre, 1 - GaussLobatto
fec = mfem.DG_FECollection(order, dim, btype=0)
fes = mfem.FiniteElementSpace(mesh, fec)
Size = fes.GetVSize()
print("Number of unknowns:", Size)
print('Number of finite element unknowns: ' + str(fes.GetVSize()))

mu_vals, w_vals = gauss_legendre_dirs(N_ang)

#ex17'den


ess_tdof_list = mfem.intArray()
dir_bdr = mfem.intArray(mesh.bdr_attributes.Max())
dir_bdr.Assign(0)
dir_bdr[3] = 1
#dir_bdr[1] = 1

S_const = 50
xs_t_const = 5.0
#xs_s_const = 1.0
S_arr = np.ones(nE) * S_const
E_arr = np.linspace(E_start, E_end, nE+1)

S_coeff = EDependentCoefficient(S_const, E_start, E_end)
xs_t_coeff = EDependentCoefficient(xs_t_const, E_start, E_end)
#xs_s_coeff = EDependentCoefficient(xs_s_const, E_start, E_end)
q_coeff = EDependentCoefficient(100, E_start, E_end)

# Solution

alpha = 1.0
beta = 0.5

psi_mu = []
psi_mu_list = []

for mu, w in zip(mu_vals, w_vals):
    print("  Solving for mu =", mu)
    mu_coeff = ConstantCoefficient(mu)
    inflow_coeff = InflowCoefficientSN(inflow, mu)
    #marker = get_marker_for_mu(mesh, mu)
    #v_coeff = MyVectorCoefficient(mu_coeff, S_coeff)
    v_coeff = mfem.VectorConstantCoefficient([mu, 0.0])
    print(f"inflow: {inflow_coeff.EvalValue([inflow, mu])}")
    #print(f"v_coeff: {v_coeff.EvalValue([mu, 0.8])}")
    print(f"sigma_t: {xs_t_coeff.EvalValue([0.0, 0.604])}")


    a = mfem.BilinearForm(fes)
    a.AddDomainIntegrator(mfem.ConvectionIntegrator(v_coeff, 1.0))
    a.AddDomainIntegrator(mfem.MassIntegrator(xs_t_coeff))
    a.AddInteriorFaceIntegrator(mfem.TransposeIntegrator(mfem.DGTraceIntegrator(v_coeff, -alpha, beta)))
    a.AddBdrFaceIntegrator(mfem.TransposeIntegrator(mfem.DGTraceIntegrator(v_coeff, -alpha, beta)), dir_bdr)
    a.Assemble()
    a.Finalize()
    A = a.SpMat()

    b = mfem.LinearForm(fes)
    b.AddDomainIntegrator(mfem.DomainLFIntegrator(q_coeff))
    b.AddBdrFaceIntegrator(mfem.BoundaryFlowIntegrator(inflow_coeff, v_coeff, -alpha, -beta), dir_bdr)
    b.Assemble()
    
    psi = mfem.GridFunction(fes)
    psi.Assign(1.0)

    prec = mfem.GSSmoother(A)
    solver = mfem.GMRESSolver()
    solver.SetOperator(A)
    solver.SetPreconditioner(prec)
    solver.SetRelTol(1e-12)
    solver.SetAbsTol(1e-12)
    solver.SetMaxIter(10000)
    solver.SetKDim(30)
    solver.SetPrintLevel(1)
    solver.Mult(b, psi)
    

    g = glvis((mesh, psi), 500,500, keys='ARjmcbp')
    g.set_size(1000, 1000)
    g.render()
    
    #X_arr = mfem.GridFunction(fes)
    #X_arr.Assign(X)
    psi.Save("psi_hs_mu_{:.3f}.gf".format(mu))
    psi_mu.append(psi.GetDataArray())
    psi_mu_list.append((mu, w, psi))


phi = mfem.GridFunction(fes)
phi.Assign(0.0)
for mu, w, X in psi_mu_list:
    phi.Add(w, X)

phi.Save("phi_dg.gf")

    

Number of unknowns: 1600
Number of finite element unknowns: 1600
  Solving for mu = -0.8611363115940526
inflow: 0
sigma_t: 5.0
   Pass :  1   Iteration :   0  ||B r|| = 2156.95
   Pass :  1   Iteration :   1  ||B r|| = 584.116
   Pass :  1   Iteration :   2  ||B r|| = 377.47
   Pass :  1   Iteration :   3  ||B r|| = 295.357
   Pass :  1   Iteration :   4  ||B r|| = 241.082
   Pass :  1   Iteration :   5  ||B r|| = 180.689
   Pass :  1   Iteration :   6  ||B r|| = 139.667
   Pass :  1   Iteration :   7  ||B r|| = 101.332
   Pass :  1   Iteration :   8  ||B r|| = 68.5894
   Pass :  1   Iteration :   9  ||B r|| = 51.4331
   Pass :  1   Iteration :  10  ||B r|| = 41.8292
   Pass :  1   Iteration :  11  ||B r|| = 38.187
   Pass :  1   Iteration :  12  ||B r|| = 32.7573
   Pass :  1   Iteration :  13  ||B r|| = 26.8563
   Pass :  1   Iteration :  14  ||B r|| = 19.913
   Pass :  1   Iteration :  15  ||B r|| = 15.2987
   Pass :  1   Iteration :  16  ||B r|| = 13.0733
   Pass :  1   Iteration :

_GlvisWidgetCore(data_str='solution\nMFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see fem/geom.hpp):\n#\n# POIN…

  Solving for mu = -0.33998104358485626
inflow: 0
sigma_t: 5.0
   Pass :  1   Iteration :   0  ||B r|| = 785.043
   Pass :  1   Iteration :   1  ||B r|| = 320.542
   Pass :  1   Iteration :   2  ||B r|| = 223.339
   Pass :  1   Iteration :   3  ||B r|| = 134.073
   Pass :  1   Iteration :   4  ||B r|| = 98.5998
   Pass :  1   Iteration :   5  ||B r|| = 84.6637
   Pass :  1   Iteration :   6  ||B r|| = 75.4213
   Pass :  1   Iteration :   7  ||B r|| = 74.1972
   Pass :  1   Iteration :   8  ||B r|| = 74.1134
   Pass :  1   Iteration :   9  ||B r|| = 71.128
   Pass :  1   Iteration :  10  ||B r|| = 64.0065
   Pass :  1   Iteration :  11  ||B r|| = 53.019
   Pass :  1   Iteration :  12  ||B r|| = 41.7318
   Pass :  1   Iteration :  13  ||B r|| = 26.0986
   Pass :  1   Iteration :  14  ||B r|| = 14.4512
   Pass :  1   Iteration :  15  ||B r|| = 7.32328
   Pass :  1   Iteration :  16  ||B r|| = 4.40354
   Pass :  1   Iteration :  17  ||B r|| = 2.59967
   Pass :  1   Iteration :  18  ||B r||

_GlvisWidgetCore(data_str='solution\nMFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see fem/geom.hpp):\n#\n# POIN…

  Solving for mu = 0.33998104358485626
inflow: 20.0
sigma_t: 5.0
   Pass :  1   Iteration :   0  ||B r|| = 871.829
   Pass :  1   Iteration :   1  ||B r|| = 446.288
   Pass :  1   Iteration :   2  ||B r|| = 308.749
   Pass :  1   Iteration :   3  ||B r|| = 208.123
   Pass :  1   Iteration :   4  ||B r|| = 133.5
   Pass :  1   Iteration :   5  ||B r|| = 97.3801
   Pass :  1   Iteration :   6  ||B r|| = 66.2294
   Pass :  1   Iteration :   7  ||B r|| = 48.6092
   Pass :  1   Iteration :   8  ||B r|| = 35.8945
   Pass :  1   Iteration :   9  ||B r|| = 26.9745
   Pass :  1   Iteration :  10  ||B r|| = 20.4583
   Pass :  1   Iteration :  11  ||B r|| = 14.0503
   Pass :  1   Iteration :  12  ||B r|| = 9.18523
   Pass :  1   Iteration :  13  ||B r|| = 5.17947
   Pass :  1   Iteration :  14  ||B r|| = 3.1562
   Pass :  1   Iteration :  15  ||B r|| = 1.5484
   Pass :  1   Iteration :  16  ||B r|| = 0.979543
   Pass :  1   Iteration :  17  ||B r|| = 0.264904
   Pass :  1   Iteration :  18  ||B r

_GlvisWidgetCore(data_str='solution\nMFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see fem/geom.hpp):\n#\n# POIN…

  Solving for mu = 0.8611363115940526
inflow: 20.0
sigma_t: 5.0
   Pass :  1   Iteration :   0  ||B r|| = 3035.62
   Pass :  1   Iteration :   1  ||B r|| = 845.42
   Pass :  1   Iteration :   2  ||B r|| = 574.063
   Pass :  1   Iteration :   3  ||B r|| = 464.275
   Pass :  1   Iteration :   4  ||B r|| = 371.297
   Pass :  1   Iteration :   5  ||B r|| = 301.644
   Pass :  1   Iteration :   6  ||B r|| = 235.402
   Pass :  1   Iteration :   7  ||B r|| = 185.717
   Pass :  1   Iteration :   8  ||B r|| = 142.623
   Pass :  1   Iteration :   9  ||B r|| = 101.147
   Pass :  1   Iteration :  10  ||B r|| = 73.0982
   Pass :  1   Iteration :  11  ||B r|| = 55.0679
   Pass :  1   Iteration :  12  ||B r|| = 47.3483
   Pass :  1   Iteration :  13  ||B r|| = 39.1216
   Pass :  1   Iteration :  14  ||B r|| = 31.2636
   Pass :  1   Iteration :  15  ||B r|| = 21.7023
   Pass :  1   Iteration :  16  ||B r|| = 15.5498
   Pass :  1   Iteration :  17  ||B r|| = 9.13632
   Pass :  1   Iteration :  18  ||B r

_GlvisWidgetCore(data_str='solution\nMFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see fem/geom.hpp):\n#\n# POIN…

In [31]:
g = glvis((mesh, phi), 500,500, keys='ARjmcbp')
g.set_size(700, 700)
g.render()

_GlvisWidgetCore(data_str='solution\nMFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see fem/geom.hpp):\n#\n# POIN…

In [32]:
g = glvis((mesh, psi), 500,500, keys='rljGacC//1')
g.set_size(900, 900)
g.render()


_GlvisWidgetCore(data_str='solution\nMFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see fem/geom.hpp):\n#\n# POIN…